In [1]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from datetime import datetime
import pandas as pd
import pandas_datareader.data as web

# replaces pyfinance.ols.PandasRollingOLS (no longer maintained)
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
idx = pd.IndexSlice

In [208]:
csvfile = r"C:\\Users\\pythonProject\\data\\data_produced\\ml4t\\20240118_industry_yf_data.csv"

raw_df = pd.read_csv(csvfile)
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2275 entries, 0 to 2274
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2275 non-null   object 
 1   XLB     2275 non-null   float64
 2   XLE     2275 non-null   float64
 3   XLF     2275 non-null   float64
 4   XLI     2275 non-null   float64
 5   XLK     2275 non-null   float64
 6   XLP     2275 non-null   float64
 7   XLU     2275 non-null   float64
 8   XLV     2275 non-null   float64
 9   XLY     2275 non-null   float64
dtypes: float64(9), object(1)
memory usage: 177.9+ KB


In [211]:
raw_df.set_index('Date',inplace=True)
raw_df.index = pd.to_datetime(raw_df.index)
raw_df

KeyError: "None of ['Date'] are in the columns"

In [139]:
# Melt the DataFrame to create the desired structure
df_transformed = raw_df.melt(id_vars='Date', var_name='Ticker', value_name='Price')
# Rearrange columns for clarity
df_transformed = df_transformed[['Ticker', 'Date', 'Price']]
df_transformed.set_index(keys = ['Date'],inplace = True)
df_transformed.index = pd.to_datetime(df_transformed.index)

In [188]:
monthly_prices = df_transformed.resample('M').last()
monthly_prices['Price'] = pd.to_numeric(monthly_prices['Price'])
monthly_prices.head()

,Ticker,Price
Date,,
2015-01-31,XLY,63.004280
2015-02-28,XLY,68.387451
2015-03-31,XLY,68.057121
2015-04-30,XLY,68.021011
2015-05-31,XLY,68.915176


In [142]:
outlier_cutoff = 0.01
data = pd.DataFrame()
lags = [1, 2, 3, 6, 9, 12]

for lag in lags:
    data['Ticker'] = monthly_prices['Ticker']
    data[f'return_{lag}m'] = (monthly_prices
                           .groupby('Ticker')['Price']
                           .pct_change(lag)
                           .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1-outlier_cutoff)))
                           .add(1)
                           .pow(1/lag)
                           .sub(1)
                           )
    
    
data

,Ticker,return_1m,return_2m,return_3m,return_6m,return_9m,return_12m
Date,,,,,,,
2015-01-31,XLY,NaN,NaN,NaN,NaN,NaN,NaN
2015-02-28,XLY,0.085441,NaN,NaN,NaN,NaN,NaN
2015-03-31,XLY,-0.004830,0.039326,NaN,NaN,NaN,NaN
2015-04-30,XLY,-0.000531,-0.002683,0.025867,NaN,NaN,NaN
2015-05-31,XLY,0.013145,0.006284,0.002566,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-09-30,XLY,-0.055320,-0.036566,-0.017060,0.013001,0.025550,0.011049
2023-10-31,XLY,-0.055162,-0.055241,-0.042805,0.005357,0.003277,0.005355
2023-11-30,XLY,0.109665,0.023940,-0.003193,0.018684,0.017369,0.012865


In [182]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start='2015')[0].drop('RF', axis=1)
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample('M').last().div(100)
factor_data.index.name = 'Date'
print(factor_data.shape,data.shape)

(107, 5)

In [183]:
factor_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 107 entries, 2015-01-31 to 2023-11-30
Freq: M
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  107 non-null    float64
 1   SMB     107 non-null    float64
 2   HML     107 non-null    float64
 3   RMW     107 non-null    float64
 4   CMA     107 non-null    float64
dtypes: float64(5)
memory usage: 5.0 KB


In [184]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 109 entries, 2015-01-31 to 2024-01-31
Freq: M
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Ticker      109 non-null    object 
 1   return_1m   108 non-null    float64
 2   return_2m   107 non-null    float64
 3   return_3m   106 non-null    float64
 4   return_6m   103 non-null    float64
 5   return_9m   100 non-null    float64
 6   return_12m  97 non-null     float64
dtypes: float64(6), object(1)
memory usage: 10.9+ KB


In [185]:
factor_data = factor_data.join(data['return_1m']).sort_index()
factor_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 107 entries, 2015-01-31 to 2023-11-30
Freq: M
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Mkt-RF     107 non-null    float64
 1   SMB        107 non-null    float64
 2   HML        107 non-null    float64
 3   RMW        107 non-null    float64
 4   CMA        107 non-null    float64
 5   return_1m  106 non-null    float64
dtypes: float64(6)
memory usage: 9.9 KB


In [176]:
grouped = factor_data.groupby('Ticker')

grouprd = grouped.apply(lambda x: RollingOLS(endog=x.return_1m,exog=sm.add_constant(x[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]),
                                     window=min(T, x.shape[0]-1)))
for name, group in grouped:
    print(name)

XLY


In [159]:
grouped_df = pd.DataFrame(grouped)
grouped_df

,0,1
0,XLY,Mkt-RF SMB HML RMW ...


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 107 entries, 2015-01-31 to 2023-11-30
Freq: M
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Mkt-RF      107 non-null    float64
 1   SMB         107 non-null    float64
 2   HML         107 non-null    float64
 3   RMW         107 non-null    float64
 4   CMA         107 non-null    float64
 5   Ticker      107 non-null    object 
 6   return_1m   106 non-null    float64
 7   return_2m   105 non-null    float64
 8   return_3m   104 non-null    float64
 9   return_6m   101 non-null    float64
 10  return_9m   98 non-null     float64
 11  return_12m  95 non-null     float64
dtypes: float64(11), object(1)
memory usage: 14.9+ KB


            Mkt-RF     SMB     HML     RMW     CMA Ticker  return_1m  \
date                                                                   
2015-01-31 -0.0311 -0.0092 -0.0358  0.0161 -0.0165    XLY        NaN   
2015-02-28  0.0613  0.0032 -0.0186 -0.0112 -0.0182    XLY   0.085441   
2015-03-31 -0.0112  0.0307 -0.0037  0.0009 -0.0052    XLY  -0.004830   
2015-04-30  0.0059 -0.0309  0.0182  0.0006 -0.0061    XLY  -0.000531   
2015-05-31  0.0136  0.0083 -0.0114 -0.0179 -0.0074    XLY   0.013145   
...            ...     ...     ...     ...     ...    ...        ...   
2023-07-31  0.0321  0.0286  0.0411 -0.0057  0.0057    XLY   0.023144   
2023-08-31 -0.0239 -0.0365 -0.0106  0.0343 -0.0237    XLY  -0.017440   
2023-09-30 -0.0524 -0.0180  0.0152  0.0186 -0.0083    XLY  -0.055320   
2023-10-31 -0.0319 -0.0404  0.0019  0.0246 -0.0066    XLY  -0.055162   
2023-11-30  0.0884 -0.0010  0.0165 -0.0389 -0.0099    XLY   0.109665   

            return_2m  return_3m  return_6m  return_9m  return_

""


In [167]:
grouped_df = grouped.apply(pd.DataFrame.reset_index)
grouped_df

Date  Mkt-RF     SMB     HML     RMW     CMA Ticker  \
Ticker                                                                 
XLY    0   2015-01-31 -0.0311 -0.0092 -0.0358  0.0161 -0.0165    XLY   
       1   2015-02-28  0.0613  0.0032 -0.0186 -0.0112 -0.0182    XLY   
       2   2015-03-31 -0.0112  0.0307 -0.0037  0.0009 -0.0052    XLY   
       3   2015-04-30  0.0059 -0.0309  0.0182  0.0006 -0.0061    XLY   
       4   2015-05-31  0.0136  0.0083 -0.0114 -0.0179 -0.0074    XLY   
...               ...     ...     ...     ...     ...     ...    ...   
       102 2023-07-31  0.0321  0.0286  0.0411 -0.0057  0.0057    XLY   
       103 2023-08-31 -0.0239 -0.0365 -0.0106  0.0343 -0.0237    XLY   
       104 2023-09-30 -0.0524 -0.0180  0.0152  0.0186 -0.0083    XLY   
       105 2023-10-31 -0.0319 -0.0404  0.0019  0.0246 -0.0066    XLY   
       106 2023-11-30  0.0884 -0.0010  0.0165 -0.0389 -0.0099    XLY   

            return_1m  return_2m  return_3m  return_6m  return_9m  return_12m  
Ticker                                                                         
XLY    0          NaN        NaN        NaN        NaN        NaN         NaN  
       1     0.085441        NaN        NaN        NaN        NaN         NaN  
       2    -0.004830   0.039326        NaN        NaN        NaN         NaN  
       3    -0.000531  -0.002683   0.025867        NaN        NaN         NaN  
       4     0.013145   0.006284   0.002566        NaN        NaN         NaN  
...               ...        ...        ...        ...        ...         ...  
       102   0.023144   0.071574   0.055943   0.027143   0.021941    0.006248  
       103  -0.017440   0.002647   0.041041   0.027808   0.018275    0.008632  
       104  -0.055320  -0.036566  -0.017060   0.013001   0.025550    0.011049  
       105  -0.055162  -0.055241  -0.042805   0.005357   0.003277    0.005355  
       106   0.109665   0.023940  -0.003193   0.018684   0.017369    0.012865  

[107 rows x 13 columns]

In [166]:
# Calculate rolling betas (corrected)
T = 24
betas = (factor_data.groupby('Ticker')
             .apply(lambda x: RollingOLS(endog=x['return_1m'],
                                         exog=sm.add_constant(x[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]),  # Exclude Ticker
                                         window=min(T, x.shape[0]-1))
                 .fit(params_only=True)
                 .params
                 .drop('const', axis=1)))
betas = betas.droplevel(1)
betas.set_index("Ticker", inplace=True)


KeyError: "None of ['Ticker'] are in the columns"

In [126]:
betas = (grouped_df.groupby('Ticker')
             .apply(lambda x: RollingOLS(endog=x['return_1m'],
                                         exog=sm.add_constant(x[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]),  # Exclude Ticker
                                         window=min(T, x.shape[0]-1))
                 .fit(params_only=True)
                 .params
                 .drop('const', axis=1)))
betas = betas.droplevel(1)
betas.set_index("Ticker", inplace=True)

KeyError: "None of ['Ticker'] are in the columns"

In [119]:
betas.describe().join(betas.sum(1).describe().to_frame('total'))

,Mkt-RF,SMB,HML,RMW,CMA,total
count,84.000000,84.000000,84.000000,84.000000,84.000000,107.000000
mean,1.087676,-0.082455,-0.191642,0.171111,-0.004533,0.769469
std,0.074581,0.111354,0.162779,0.276016,0.370813,0.576461
min,0.970311,-0.331319,-0.613484,-0.280135,-0.808329,0.000000
25%,1.019704,-0.157225,-0.267458,-0.055752,-0.124659,0.160349
50%,1.081588,-0.087989,-0.187491,0.161439,-0.026881,0.853825
75%,1.145527,0.008451,-0.121106,0.296693,0.210297,1.294716
max,1.309687,0.157884,0.152536,1.026903,0.840457,1.748976


In [125]:
grouped_df

,date,Mkt-RF,SMB,HML,RMW,CMA,return_1m,return_2m,return_3m,return_6m,return_9m,return_12m
Ticker,,,,,,,,,,,,
XLY,2015-01-31,-0.0311,-0.0092,-0.0358,0.0161,-0.0165,NaN,NaN,NaN,NaN,NaN,NaN
XLY,2015-02-28,0.0613,0.0032,-0.0186,-0.0112,-0.0182,0.085441,NaN,NaN,NaN,NaN,NaN
XLY,2015-03-31,-0.0112,0.0307,-0.0037,0.0009,-0.0052,-0.004830,0.039326,NaN,NaN,NaN,NaN
XLY,2015-04-30,0.0059,-0.0309,0.0182,0.0006,-0.0061,-0.000531,-0.002683,0.025867,NaN,NaN,NaN
XLY,2015-05-31,0.0136,0.0083,-0.0114,-0.0179,-0.0074,0.013145,0.006284,0.002566,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
XLY,2023-07-31,0.0321,0.0286,0.0411,-0.0057,0.0057,0.023144,0.071574,0.055943,0.027143,0.021941,0.006248
XLY,2023-08-31,-0.0239,-0.0365,-0.0106,0.0343,-0.0237,-0.017440,0.002647,0.041041,0.027808,0.018275,0.008632
XLY,2023-09-30,-0.0524,-0.0180,0.0152,0.0186,-0.0083,-0.055320,-0.036566,-0.017060,0.013001,0.025550,0.011049


In [122]:
data = (grouped_df
        .join(betas
              .groupby(level='Ticker')
              .shift()))
data.info()

ValueError: columns overlap but no suffix specified: Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA'], dtype='object')

In [72]:
# Sample DataFrame with a single-level DatetimeIndex
df = pd.DataFrame({'Ticker': ['A', 'A', 'B', 'B', 'A'],
                   'Value': [10, 20, 30, 40, 50]},
                   index=pd.date_range('2023-01-01', periods=5))

grouped = df.groupby('Ticker')
# Group by Ticker without creating a new index level
for name, group in grouped:
    print(name)  # Prints the Ticker name
    print(group)  # Prints the DataFrame for that Ticker



A
           Ticker  Value
2023-01-01      A     10
2023-01-02      A     20
2023-01-05      A     50
B
           Ticker  Value
2023-01-03      B     30
2023-01-04      B     40
